# Setup, Constants, and Imports

In [1]:
import os
import sys
import logging
import pandas as pd
import numpy as np
import pandas as pd
import spacy
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 2.7 MB/s eta 0:00:05
     --- ------------------------------------ 1.0/12.8 MB 2.8 MB/s eta 0:00:05
     ---- ----------------------------------- 1.6/12.8 MB 2.5 MB/s eta 0:00:05
     ------ --------------------------------- 2.1/12.8 MB 2.6 MB/s eta 0:00:05
     -------- ------------------------------- 2.6/12.8 MB 2.5 MB/s eta 0:00:05
     --------- ------------------------------ 2.9/12.8 MB 2.4 MB/s eta 0:00:05
     --------- ------------------------------ 3.1/12.8 MB 2.2 MB/s eta 0:00:05
     ---------- ----------------------------- 3.4/12.8 MB 2.1 MB/s eta 0:00:05
     ---------- ----------------------------- 3.4/12.8 MB 2.1 MB/s eta 0:00:05
     ---------- ----------------------------- 3.4/12.8 MB 2.1 MB/s eta 0:00:05
     ----------- ---------------------------- 3.7/12.8 MB 1.6 MB/s

## Notebook Configs

In [3]:
IS_COLAB = 'google.colab' in sys.modules
OUTPUT_PROCESSED_FILES = False # TODO: Use this if you want to output save files (optional - see below)

if IS_COLAB:
    from google.colab import userdata
    GITHUB_USERNAME = userdata.get('github_user')
    GITHUB_TOKEN = userdata.get('github_token')
    GITHUB_EMAIL = userdata.get('github_email')

## Constants

In [4]:
REPO_URL = "https://github.com/EErlando/Quarterly-Bytes.git"
REPO_NAME = "src"
REPO_BRANCH = "LP_topic_modelling_extended" # TODO: UPDATE THIS TO YOU BRANCH - DEFAULT TO MAIN
NOTEBOOK_DIR = "3_modelling" # TODO: UPDATE THIS TO YOUR NOTEBOOK DIRECTORY (e.g. 1_data_extraction_and_processing)

## Clone and Pull Latest from Repository - Colab Specific

In [5]:
if IS_COLAB:
    !git config pull.rebase false
    if os.path.exists(REPO_NAME):
        print(f"Directory '{REPO_NAME}' already exists. Pulling latest changes...")
        %cd {REPO_NAME}
        !git pull origin {REPO_BRANCH} --quiet
        %cd ..
    else:
        print(f"Cloning repository into '{REPO_NAME}'...")
        !git clone --quiet --branch {REPO_BRANCH} {REPO_URL} {REPO_NAME}
        print("Clone complete.")

    sys.path.append('/content/src/')
    %cd /content/src/
    !pip install -r requirements.txt
else:
    if os.path.basename(os.getcwd()) == NOTEBOOK_DIR:
        os.chdir('../../') # TODO: UPDATE THIS TO ROOT OF REPO
    
    !pip install -r requirements.txt

logging.basicConfig(level=logging.ERROR, format='%(levelname)s: %(message)s')

  Using cached bertopic-0.17.0-py3-none-any.whl.metadata (23 kB)
  Using cached umap_learn-0.5.7-py3-none-any.whl.metadata (21 kB)
  Using cached hdbscan-0.8.40.tar.gz (6.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached plotly-6.1.2-py3-none-any.whl.metadata (6.9 kB)
  Using cached numba-0.61.2-cp313-cp313-win_amd64.whl.metadata (2.8 kB)
  Using cached pynndescent-0.5.13-py3-none-any.whl.metadata (6.8 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'do

ERROR: To modify pip, please run the following command:
C:\Users\zifen\AppData\Local\Programs\Python\Python313\python.exe -m pip install -r requirements.txt


## Local Imports

In [6]:
from src.utils.common_helpers import read_yaml_file
from src.utils.common_helpers import read_list_from_text_file

## Helpers

In [7]:
def group_df(df, group_by_columns, agg_column='content'):
    """
    Groups the DataFrame by specified columns and aggregates the content column.
    
    Parameters:
    - df: DataFrame to group
    - group_by_columns: List of columns to group by
    - agg_column: Column to aggregate (default is 'content')
    
    Returns:
    - Grouped DataFrame with aggregated content
    """
    return df.groupby(group_by_columns, as_index=False).agg({agg_column: ' '.join})


## Load Data

In [8]:
gs_discussion_df = pd.read_csv('data/processed/Goldman Sachs/discussion_df.csv')
gs_qna_df = pd.read_csv('data/processed/Goldman Sachs/qna_df.csv')
jp_discussion_df = pd.read_csv('data/processed/JP Morgan/discussion_df.csv')
jp_qna_df = pd.read_csv('data/processed/JP Morgan/qna_df.csv')


# Goldman Sachs
grouped_gs_discussion_df = group_df(gs_discussion_df, ['quarter', 'year'])
grouped_gs_qna_df = group_df(gs_qna_df, ['question_answer_group_id', 'quarter', 'year'])

# JP Morgan
grouped_jp_discussion_df = group_df(jp_discussion_df, ['quarter', 'year'])
grouped_jp_qna_df = group_df(jp_qna_df, ['question_answer_group_id', 'quarter', 'year'])


# Topic Modelling

In [9]:
gs_stopwords = set(read_list_from_text_file('src/data_processing/goldman_sachs_topic_modelling_stopwords.txt'))
abbreviations = read_yaml_file('src/abbreviations.yaml')

In [10]:


try:
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
except OSError:
    print("SpaCy 'en_core_web_sm' model not found. Please run: python -m spacy download en_core_web_sm")
    exit()

all_stop_words = nlp.Defaults.stop_words.union(gs_stopwords)

def preprocess_text(text: str, stop_words: set, abbreviations: dict) -> str:
    if not isinstance(text, str):
        return ""
    
    processed_text = text.lower()
    processed_text = re.sub(r'[-_]+', ' ', processed_text).strip()
    
    sorted_phrases = sorted(abbreviations.items(), key=lambda item: len(item[1]), reverse=True)
    
    for abbrev, phrase in sorted_phrases:
        processed_text = re.sub(r'\b' + re.escape(phrase.lower()) + r'\b', abbrev.lower(), processed_text)

    processed_text = re.sub(r'\b\d+\b', '', processed_text).strip()

    doc = nlp(processed_text)

    tokens = []
    for token in doc:
        if token.text not in stop_words or token.text in abbreviations.keys():
            tokens.append(token.lemma_) # Lemmatize the token (abbreviations won't change)

    return " ".join(tokens)


In [14]:
!pip install bertopic 

from bertopic import BERTopic


  Using cached bertopic-0.17.0-py3-none-any.whl.metadata (23 kB)
  Using cached hdbscan-0.8.40.tar.gz (6.9 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached plotly-6.1.2-py3-none-any.whl.metadata (6.9 kB)
  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached umap_learn-0.5.7-py3-none-any.whl.metadata (21 kB)
  Using cached narwhals-1.42.1-py3-none-any.whl.metadata (11 kB)
  Using cached numba-0.61.2-cp313-cp313-win_amd64.whl.metadata (2.8 kB)
  Using cached pynndescent-0.5.13-py3-none-any.whl.metadata (6.8 kB)
  Using cached llvmlite-0.44.0-cp313-cp313-win_amd64.whl.metadata (5.0 kB)
  Using cached numpy-2.2.6-cp313-cp313-win_amd64.whl.metadata (60 kB)
Us

  error: subprocess-exited-with-error
  
  × Building wheel for hdbscan (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [39 lines of output]
      C:\Users\zifen\AppData\Local\Temp\pip-build-env-qwvygn_c\overlay\Lib\site-packages\setuptools\_distutils\dist.py:289: UserWarning: Unknown distribution option: 'test_suite'
        warnings.warn(msg)
      C:\Users\zifen\AppData\Local\Temp\pip-build-env-qwvygn_c\overlay\Lib\site-packages\setuptools\_distutils\dist.py:289: UserWarning: Unknown distribution option: 'tests_require'
        warnings.warn(msg)
      C:\Users\zifen\AppData\Local\Temp\pip-build-env-qwvygn_c\overlay\Lib\site-packages\setuptools\dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license expression:
      
              License :: 

ModuleNotFoundError: No module named 'bertopic'

In [11]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
import os

# Conditional imports for BERTopic
try:
    from bertopic import BERTopic
    from umap import UMAP
    import hdbscan
    from sentence_transformers import SentenceTransformer
    BERTOPIC_AVAILABLE = True
except ImportError:
    print("BERTopic, UMAP, HDBSCAN, or Sentence Transformers not found. BERTopic functionality will be disabled.")
    BERTOPIC_AVAILABLE = False

# --- Mock Preprocessing Function (replace with your actual preprocess_text) ---
def preprocess_text(text, stop_words=None, abbreviations=None):
    """
    A mock preprocessing function to clean and prepare text.
    In a real application, this would include tokenization, stemming/lemmatization,
    punctuation removal, number handling, etc.
    """
    text = str(text).lower()
    # Simple tokenization and stop word removal for demonstration
    words = text.split()
    if stop_words:
        words = [word for word in words if word not in stop_words]
    # Simple abbreviation handling (example: 'ai' -> 'artificial intelligence')
    if abbreviations:
        for abbr, full in abbreviations.items():
            words = [full if word == abbr else word for word in words]
    return " ".join(words)

# --- Custom Transformer for Text Preprocessing ---
class TextPreprocessor(BaseEstimator, TransformerMixin):
    """
    A custom scikit-learn transformer to apply text preprocessing.
    It wraps the 'preprocess_text' function.
    """
    def __init__(self, stop_words=None, abbreviations=None):
        self.stop_words = stop_words
        self.abbreviations = abbreviations

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        print("Starting Phase 1: Preprocessing...")
        preprocessed_X = [preprocess_text(text, self.stop_words, self.abbreviations) for text in X]
        print("Preprocessing complete.")
        return pd.Series(preprocessed_X)

# --- Custom Estimator for LDA Topic Modeling ---
class LDATopicModeler(BaseEstimator, TransformerMixin):
    """
    A custom scikit-learn estimator that wraps LatentDirichletAllocation.
    Allows for fitting and transforming data using an LDA model within a pipeline.
    """
    def __init__(self, n_components=10, max_iter=10, learning_method='online',
                 learning_decay=0.7, random_state=42, n_jobs=-1, evaluate_every=10):
        self.n_components = n_components
        self.max_iter = max_iter
        self.learning_method = learning_method
        self.learning_decay = learning_decay
        self.random_state = random_state
        self.n_jobs = n_jobs
        self.evaluate_every = evaluate_every
        self.lda_model = None

    def fit(self, X, y=None):
        print(f"\nStarting Phase 3: Topic Modeling (LDA) with {self.n_components} topics...")
        self.lda_model = LatentDirichletAllocation(
            n_components=self.n_components,
            max_iter=self.max_iter,
            learning_method=self.learning_method,
            learning_decay=self.learning_decay,
            random_state=self.random_state,
            n_jobs=self.n_jobs,
            evaluate_every=self.evaluate_every
        )
        self.lda_model.fit(X)
        print(f"LDA model fitting complete for {self.n_components} topics.")
        return self

    def transform(self, X):
        if self.lda_model is None:
            raise RuntimeError("LDA model not fitted. Call fit() first.")
        print(f"Transforming data with fitted LDA model (k={self.n_components})...")
        topic_distribution = self.lda_model.transform(X)
        print("Transformation complete.")
        return topic_distribution

    def get_model(self):
        return self.lda_model

# --- Custom Estimator for BERTopic Modeling ---
class BERTopicWrapper(BaseEstimator, TransformerMixin):
    """
    A custom scikit-learn estimator that wraps BERTopic.
    """
    def __init__(self, embedding_model='all-MiniLM-L6-v2', umap_args=None, hdbscan_args=None,
                 vectorizer_args=None, nr_topics="auto", calculate_probabilities=True, **bertopic_kwargs):
        if not BERTOPIC_AVAILABLE:
            raise ImportError("BERTopic and its dependencies are required for BERTopicWrapper.")

        self.embedding_model_name = embedding_model
        self.umap_args = umap_args if umap_args is not None else {}
        self.hdbscan_args = hdbscan_args if hdbscan_args is not None else {}
        self.vectorizer_args = vectorizer_args if vectorizer_args is not None else {}
        self.nr_topics = nr_topics
        self.calculate_probabilities = calculate_probabilities
        self.bertopic_kwargs = bertopic_kwargs
        self.bertopic_model = None

    def fit(self, X, y=None):
        print("\nStarting Phase 3: Topic Modeling (BERTopic)...")

        # Initialize UMAP and HDBSCAN models
        umap_model = UMAP(**self.umap_args)
        hdbscan_model = hdbscan.HDBSCAN(
            min_cluster_size=10,  # Default, can be overridden by hdbscan_args
            metric='euclidean',
            cluster_selection_method='eom',
            prediction_data=True, # Required for transform to assign topics to new data
            **self.hdbscan_args
        )
        # Initialize CountVectorizer for BERTopic (can be customized)
        vectorizer_model = TfidfVectorizer(
            min_df=10,  # Default, can be overridden by vectorizer_args
            ngram_range=(1, 3),
            **self.vectorizer_args
        )

        # Initialize SentenceTransformer
        embedding_model = SentenceTransformer(self.embedding_model_name)

        self.bertopic_model = BERTopic(
            embedding_model=embedding_model,
            umap_model=umap_model,
            hdbscan_model=hdbscan_model,
            vectorizer_model=vectorizer_model,
            nr_topics=self.nr_topics,
            calculate_probabilities=self.calculate_probabilities,
            **self.bertopic_kwargs
        )

        # X is expected to be a pandas Series of preprocessed text
        self.topics, self.probs = self.bertopic_model.fit_transform(X.tolist())
        print("BERTopic model fitting complete.")
        return self

    def transform(self, X):
        if self.bertopic_model is None:
            raise RuntimeError("BERTopic model not fitted. Call fit() first.")
        print("Transforming data with fitted BERTopic model...")
        topics, probs = self.bertopic_model.transform(X.tolist())
        print("Transformation complete.")
        return topics # Return topic assignments

    def get_model(self):
        return self.bertopic_model

# --- Utility function to display topics (adapted for both LDA and BERTopic) ---
def display_topics(model, vectorizer=None, no_top_words=10, file=None, model_type='lda'):
    """
    Prints or writes the top words for each topic.
    Args:
        model: The fitted topic model (LDA or BERTopic).
        vectorizer (TfidfVectorizer, optional): The fitted TF-IDF vectorizer (for LDA).
        no_top_words (int): The number of top words to display for each topic.
        file (file object, optional): If provided, topics will be written to this file.
        model_type (str): 'lda' or 'bertopic' to specify model type for appropriate display.
    """
    if model_type == 'lda':
        if vectorizer is None:
            raise ValueError("Vectorizer must be provided for LDA topic display.")
        feature_names = vectorizer.get_feature_names_out()
        for topic_idx, topic in enumerate(model.components_):
            topic_words = " ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])
            output_str = f"Topic {topic_idx + 1}:\n{topic_words}\n"
            if file:
                file.write(output_str)
            else:
                print(output_str)
    elif model_type == 'bertopic':
        if not BERTOPIC_AVAILABLE:
            print("BERTopic not available, cannot display BERTopic topics.")
            return

        topic_info = model.get_topic_info()
        output_str = "\nBERTopic - Top Words per Topic:\n"
        if file:
            file.write(output_str)
        else:
            print(output_str)

        # Iterate through all topics, excluding the noise topic (-1)
        for topic_id in topic_info.Topic.unique():
            if topic_id == -1: # Skip noise topic
                continue
            # Get the top words for the current topic
            words = model.get_topic(topic_id)
            if words:
                top_words = ", ".join([word for word, _ in words[:no_top_words]])
                topic_name = topic_info[topic_info['Topic'] == topic_id]['Name'].iloc[0]
                output_str = f"Topic {topic_id} ({topic_name}): {top_words}\n"
                if file:
                    file.write(output_str)
                else:
                    print(output_str)
            else:
                output_str = f"Topic {topic_id}: No words found.\n"
                if file:
                    file.write(output_str)
                else:
                    print(output_str)
    else:
        raise ValueError("Invalid model_type. Choose 'lda' or 'bertopic'.")


# --- Main Topic Modeling Pipeline Class ---
class TopicModelingPipeline:
    def __init__(self, model_type='lda', **kwargs):
        """
        Initializes the topic modeling pipeline.

        Args:
            model_type (str): The type of topic model to use ('lda' or 'bertopic').
            **kwargs: Arguments specific to the chosen model or pipeline steps.
                      For LDA: max_df, min_df, ngram_range (for TF-IDF), n_components, max_iter, etc.
                      For BERTopic: embedding_model, umap_args, hdbscan_args, vectorizer_args, nr_topics, etc.
        """
        self.model_type = model_type
        self.pipeline = self._build_pipeline(**kwargs)

    def _build_pipeline(self, **kwargs):
        """Builds the scikit-learn pipeline based on the specified model_type."""
        preprocessor_kwargs = {
            'stop_words': kwargs.pop('stop_words', ['a', 'the', 'is', 'and', 'of', 'to', 'in', 'for']),
            'abbreviations': kwargs.pop('abbreviations', {'ai': 'artificial intelligence', 'ml': 'machine learning'})
        }

        pipeline_steps = [
            ('preprocessor', TextPreprocessor(**preprocessor_kwargs))
        ]

        if self.model_type == 'lda':
            tfidf_kwargs = {
                'max_df': kwargs.pop('max_df', 0.95),
                'min_df': kwargs.pop('min_df', 2), # Adjusted for dummy data
                'ngram_range': kwargs.pop('ngram_range', (1, 3))
            }
            lda_kwargs = {
                'n_components': kwargs.pop('n_components', 5), # Default LDA topics
                'random_state': kwargs.pop('random_state', 42),
                'max_iter': kwargs.pop('max_iter', 10),
                'learning_method': kwargs.pop('learning_method', 'online'),
                'learning_decay': kwargs.pop('learning_decay', 0.7),
                'n_jobs': kwargs.pop('n_jobs', -1),
                'evaluate_every': kwargs.pop('evaluate_every', 10)
            }
            pipeline_steps.append(('tfidf_vectorizer', TfidfVectorizer(**tfidf_kwargs)))
            pipeline_steps.append(('topic_modeler', LDATopicModeler(**lda_kwargs)))
        elif self.model_type == 'bertopic':
            if not BERTOPIC_AVAILABLE:
                raise ImportError("BERTopic and its dependencies must be installed to use model_type='bertopic'.")

            bertopic_kwargs = {
                'embedding_model': kwargs.pop('embedding_model', 'all-MiniLM-L6-v2'),
                'umap_args': kwargs.pop('umap_args', {}),
                'hdbscan_args': kwargs.pop('hdbscan_args', {}),
                'vectorizer_args': kwargs.pop('vectorizer_args', {}),
                'nr_topics': kwargs.pop('nr_topics', "auto"),
                'calculate_probabilities': kwargs.pop('calculate_probabilities', True),
                **kwargs # Pass any remaining kwargs directly to BERTopicWrapper
            }
            pipeline_steps.append(('topic_modeler', BERTopicWrapper(**bertopic_kwargs)))
        else:
            raise ValueError(f"Unknown model_type: {self.model_type}. Choose 'lda' or 'bertopic'.")

        # Any remaining kwargs are ignored if not consumed by model-specific initializations
        if kwargs:
            print(f"Warning: Unused keyword arguments passed to pipeline: {kwargs}")

        return Pipeline(pipeline_steps)

    def fit(self, X, y=None):
        """Fits the entire pipeline to the input data."""
        print(f"\n--- Fitting {self.model_type.upper()} Topic Modeling Pipeline ---")
        self.pipeline.fit(X, y)
        return self

    def transform(self, X):
        """Transforms the input data and returns topic assignments/distributions."""
        return self.pipeline.transform(X)

    def get_topic_model(self):
        """Returns the underlying fitted topic model (LDA or BERTopic)."""
        return self.pipeline.named_steps['topic_modeler'].get_model()

    def get_vectorizer(self):
        """Returns the fitted vectorizer (TF-IDF for LDA, None for BERTopic)."""
        if self.model_type == 'lda':
            return self.pipeline.named_steps['tfidf_vectorizer']
        return None

# --- Demonstration with Dummy Data ---
if __name__ == "__main__":
    # Create a dummy DataFrame similar to your 'grouped_gs_qna_df'
    dummy_data = {
        'content': [
            "This is a document about machine learning and artificial intelligence.",
            "Natural language processing is a fascinating field.",
            "Deep learning models are used in computer vision.",
            "Another document discussing data science and big data analytics.",
            "Quantum computing is a cutting-edge area of research.",
            "This document talks about machine learning algorithms and data analysis.",
            "Artificial intelligence applications in healthcare are growing.",
            "Big data analytics helps in making informed decisions.",
            "Exploring the world of natural language generation.",
            "A comprehensive guide to computer vision techniques."
        ],
        'other_column': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    }
    # grouped_gs_qna_df = pd.DataFrame(dummy_data)

    output_dir = "data/temp/leslie_topic_modelling_fine_tuning"
    os.makedirs(output_dir, exist_ok=True)

    no_top_words = 10

    # --- Run with LDA ---
    print("\n\n##################################")
    print("### Running Topic Modeling with LDA ###")
    print("##################################")

    lda_pipeline_instance = TopicModelingPipeline(
        model_type='lda',
        n_components=5, # Number of topics for LDA
        min_df=2 # Adjusted min_df for dummy data
    )
    lda_pipeline_instance.fit(grouped_gs_qna_df['content'])

    lda_model = lda_pipeline_instance.get_topic_model()
    tfidf_vectorizer = lda_pipeline_instance.get_vectorizer()

    output_filename_lda = f"{output_dir}/lda_topics_k{lda_model.n_components}.txt"
    with open(output_filename_lda, 'w', encoding='utf-8') as f:
        f.write(f"--- LDA Topic Model with {lda_model.n_components} Topics ---\n\n")
        f.write("Interpreting Topics:\n")
        display_topics(lda_model, tfidf_vectorizer, no_top_words, file=f, model_type='lda')
    print(f"LDA Topics saved to {output_filename_lda}")

    # Assign dominant topics for LDA
    lda_topic_assignments = lda_pipeline_instance.transform(grouped_gs_qna_df['content'])
    grouped_gs_qna_df[f'dominant_topic_lda_k{lda_model.n_components}'] = np.argmax(lda_topic_assignments, axis=1)

    print(f"\nExample (LDA): Documents with their dominant topic (first 5 rows) for k={lda_model.n_components}:")
    for i in range(min(5, len(grouped_gs_qna_df))):
        original_content = grouped_gs_qna_df.loc[i, 'content']
        truncated_content = (original_content[:100] + '...') if len(original_content) > 100 else original_content
        dominant_topic = grouped_gs_qna_df.loc[i, f'dominant_topic_lda_k{lda_model.n_components}']
        print(f"Doc {i}: Topic {dominant_topic} - Content: \"{truncated_content}\"")
    print("\nUpdated DataFrame (LDA results - first 5 rows):")
    print(grouped_gs_qna_df.head())


    # --- Run with BERTopic ---
    if BERTOPIC_AVAILABLE:
        print("\n\n#####################################")
        print("### Running Topic Modeling with BERTopic ###")
        print("#####################################")

        bertopic_pipeline_instance = TopicModelingPipeline(
            model_type='bertopic',
            nr_topics="auto", # Let BERTopic determine number of topics
            calculate_probabilities=True,
            # You can pass BERTopic specific arguments here:
            # umap_args={'n_neighbors': 15, 'n_components': 5},
            # hdbscan_args={'min_cluster_size': 10},
            # vectorizer_args={'min_df': 5}
        )
        bertopic_pipeline_instance.fit(grouped_gs_qna_df['content'])

        bertopic_model = bertopic_pipeline_instance.get_topic_model()

        output_filename_bertopic = f"{output_dir}/bertopic_topics.txt"
        with open(output_filename_bertopic, 'w', encoding='utf-8') as f:
            f.write("--- BERTopic Model ---\n\n")
            f.write("Interpreting Topics:\n")
            display_topics(bertopic_model, no_top_words=no_top_words, file=f, model_type='bertopic')
        print(f"BERTopic Topics saved to {output_filename_bertopic}")

        # Assign dominant topics for BERTopic
        bertopic_topic_assignments = bertopic_pipeline_instance.transform(grouped_gs_qna_df['content'])
        grouped_gs_qna_df['dominant_topic_bertopic'] = bertopic_topic_assignments

        print("\nExample (BERTopic): Documents with their dominant topic (first 5 rows):")
        for i in range(min(5, len(grouped_gs_qna_df))):
            original_content = grouped_gs_qna_df.loc[i, 'content']
            truncated_content = (original_content[:100] + '...') if len(original_content) > 100 else original_content
            dominant_topic = grouped_gs_qna_df.loc[i, 'dominant_topic_bertopic']
            print(f"Doc {i}: Topic {dominant_topic} - Content: \"{truncated_content}\"")
        print("\nUpdated DataFrame (BERTopic results - first 5 rows):")
        print(grouped_gs_qna_df.head())
    else:
        print("\nSkipping BERTopic demonstration as required libraries are not installed.")

    print("\nOverall processing complete. Check the 'data/temp/leslie_topic_modelling_fine_tuning' directory for output files.")


BERTopic, UMAP, HDBSCAN, or Sentence Transformers not found. BERTopic functionality will be disabled.


##################################
### Running Topic Modeling with LDA ###
##################################

--- Fitting LDA Topic Modeling Pipeline ---
Starting Phase 1: Preprocessing...
Preprocessing complete.

Starting Phase 3: Topic Modeling (LDA) with 5 topics...


KeyboardInterrupt: 

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

def vectorize_text(data_series, max_df=0.95, min_df=5, stop_words=None, ngram_range=(1, 1)):
    """
    Performs TF-IDF vectorization on a given text series.
    Args:
        data_series (pd.Series): A pandas Series containing preprocessed text content.
        max_df (float): When building the vocabulary ignore terms that have a document
                        frequency strictly higher than the given threshold (corpus-specific stop words).
        min_df (int): When building the vocabulary ignore terms that have a document
                      frequency strictly lower than the given threshold.
        stop_words (str or list, optional): 'english' for a built-in stop word list,
                                            a list of custom stop words, or None.
    Returns:
        tuple: A tuple containing:
            - dtm (scipy.sparse.csr_matrix): The Document-Term Matrix.
            - vectorizer (TfidfVectorizer): The fitted TF-IDF vectorizer.
    """
    print("\nStarting Phase 3: Vectorization (TF-IDF)...")
    vectorizer = TfidfVectorizer(max_df=max_df, min_df=min_df, stop_words=stop_words, ngram_range=ngram_range)
    dtm = vectorizer.fit_transform(data_series)
    print(f"TF-IDF Vectorization complete. Document-Term Matrix shape: {dtm.shape}")
    return dtm, vectorizer

def display_topics(model, feature_names, no_top_words, file=None):
    """
    Prints or writes the top words for each topic from an LDA model.
    Args:
        model (LatentDirichletAllocation): The fitted LDA model.
        feature_names (list): List of feature names (words) from the vectorizer.
        no_top_words (int): The number of top words to display for each topic.
        file (file object, optional): If provided, topics will be written to this file.
                                      Otherwise, they will be printed to the console.
    """
    for topic_idx, topic in enumerate(model.components_):
        topic_words = " ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])
        if file:
            file.write(f"\nTopic {topic_idx + 1}:\n")
            file.write(f"{topic_words}\n")
        else:
            print(f"\nTopic {topic_idx + 1}:")
            print(topic_words)

def run_lda_topic_modeling(dtm, vectorizer, num_topics_range, output_dir="data/temp/leslie_topic_modelling_fine_tuning", no_top_words=10, df_to_update=None):
    """
    Performs LDA topic modeling for a range of topic numbers and saves the results.
    Args:
        dtm (scipy.sparse.csr_matrix): The Document-Term Matrix from TF-IDF vectorization.
        vectorizer (TfidfVectorizer): The fitted TF-IDF vectorizer.
        num_topics_range (list): A list of integers representing the number of topics to experiment with.
        output_dir (str): Directory to save the topic modeling output files.
        no_top_words (int): The number of top words to display for each topic.
        df_to_update (pd.DataFrame, optional): The DataFrame to which dominant topics will be
                                                assigned. If None, dominant topics won't be added.
    """
    print("\nStarting Phase 4: Topic Modeling (LDA) for a range of topics...")

    feature_names = vectorizer.get_feature_names_out()

    for num_topics in num_topics_range:
        output_filename = f"{output_dir}/lda_topics_k{num_topics}.txt"
        print(f"\n--- Processing with {num_topics} topics. Outputting to {output_filename} ---")

        with open(output_filename, 'w', encoding='utf-8') as f:
            f.write(f"--- LDA Topic Model with {num_topics} Topics ---\n\n")

            lda = LatentDirichletAllocation(n_components=num_topics,
                                            max_iter=10,
                                            learning_method='online',
                                            learning_decay=0.7,
                                            random_state=42,
                                            n_jobs=-1,
                                            evaluate_every=10)

            lda.fit(dtm)
            f.write(f"LDA model fitting complete for {num_topics} topics.\n\n")

            f.write("\nInterpreting Topics:\n")
            display_topics(lda, feature_names, no_top_words, file=f)

            if df_to_update is not None:
                topic_distribution = lda.transform(dtm)
                df_to_update[f'dominant_topic_k{num_topics}'] = np.argmax(topic_distribution, axis=1)

                print(f"\nExample: Documents with their dominant topic (first 5 rows) for k={num_topics}:")
                for i in range(min(5, len(df_to_update))):
                    original_content = df_to_update.loc[i, 'content']
                    truncated_content = (original_content[:200] + '...') if len(original_content) > 200 else original_content
                    dominant_topic = df_to_update.loc[i, f'dominant_topic_k{num_topics}']
                    print(f"Doc {i}: Topic {dominant_topic} - Content: \"{truncated_content}\"")

grouped_gs_qna_df['preprocessed_content'] = grouped_gs_qna_df['content'].apply(lambda x: preprocess_text(x, all_stop_words, abbreviations))
dtm_matrix, tfidf_vectorizer = vectorize_text(grouped_gs_qna_df['preprocessed_content'], ngram_range=(1, 3))

# num_topics_range = [8, 9, 10]
# num_topics_range = [2, 5, 8, 11, 14, 17, 20, 23]
num_topics_range = [5, 8, 11, 14]
run_lda_topic_modeling(dtm_matrix, tfidf_vectorizer, num_topics_range, df_to_update=grouped_gs_qna_df, )

print("\nProcessing complete. Check the 'data/temp/leslie_topic_modelling_fine_tuning' directory for output files.")
print("\nUpdated DataFrame with dominant topics (first 5 rows):")
print(grouped_gs_qna_df.head())

In [ ]:
# (Conceptual - requires FinBERT model loading and appropriate tokenization)

import pandas as pd
from transformers import AutoTokenizer, AutoModel
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
def get_finbert_embeddings(texts):
    tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
    model = AutoModel.from_pretrained("ProsusAI/finbert")

    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the [CLS] token embedding as the sentence embedding
    return outputs.last_hidden_state[:, 0, :].numpy()

def run_finbert_topic_clustering(df, num_topics_range, output_dir="data/temp/finbert_topic_clustering"):
    print("\nStarting Topic Clustering using FinBERT embeddings...")

    # 1. Get FinBERT embeddings for your documents
    document_embeddings = get_finbert_embeddings(df['content'].tolist())
    print(f"Generated {document_embeddings.shape[0]} document embeddings of size {document_embeddings.shape[1]}.")

    for num_topics in num_topics_range:
        print(f"\n--- Processing with {num_topics} clusters (topics) using FinBERT embeddings ---")
        kmeans = KMeans(n_clusters=num_topics, random_state=42, n_init=10) # n_init for stability
        kmeans.fit(document_embeddings)
        df[f'finbert_dominant_topic_k{num_topics}'] = kmeans.labels_

        # 2. Interpret topics (example: finding top words per cluster using TF-IDF on cluster documents)
        print(f"\nInterpreting Clusters for k={num_topics}:")
        for i in range(num_topics):
            cluster_docs = df[df[f'finbert_dominant_topic_k{num_topics}'] == i]['content']
            if len(cluster_docs) > 0:
                # Use TF-IDF to find important words in this cluster
                tfidf_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
                tfidf_matrix = tfidf_vectorizer.fit_transform(cluster_docs)
                feature_names = tfidf_vectorizer.get_feature_names_out()
                sums_tfidf = tfidf_matrix.sum(axis=0)
                # Get top words
                top_words_indices = sums_tfidf.argsort()[0, ::-1][:10]
                top_words = [feature_names[idx] for idx in top_words_indices.tolist()[0]]
                print(f"Cluster {i} (Top Words): {', '.join(top_words)}")
            else:
                print(f"Cluster {i}: No documents in this cluster.")

        print(f"\nExample: Documents with their dominant topic (first 5 rows) for k={num_topics}:")
        for i in range(min(5, len(df))):
            original_content = df.loc[i, 'content']
            truncated_content = (original_content[:200] + '...') if len(original_content) > 200 else original_content
            dominant_topic = df.loc[i, f'finbert_dominant_topic_k{num_topics}']
            print(f"Doc {i}: Topic {dominant_topic} - Content: \"{truncated_content}\"")

    return df

# To run this, you would need to:
# 1. Install transformers and torch: pip install transformers torch scikit-learn pandas
# 2. Add the actual run_finbert_topic_clustering call with your dtm and df_to_update
# For example: df_to_update = run_finbert_topic_clustering(df_to_update.copy(), num_topics_range=[2, 3])

Conclusion and Recommendation
The addition of stopwords has certainly helped in some areas, making certain topics clearer. However, some conversational noise still persists, especially words related to the Q&A format or general conversational patterns. The term "Apple" continues to be grouped with "deposit" in some k values, which is still a bit puzzling without specific context.

Based on this comprehensive analysis, the most sensible k value is a trade-off between granularity, coherence, and minimizing "junk" topics.

k=8: Offers good clarity for key themes (Credit Card, Headcount/Severance, Asset/Fundraising), but is still quite broad and has some remaining conversational noise.
k=9: Introduces very strong "GSIB" and "Wealth Management" topics.
k=10: Shows strong "Investment/Platform" and "Fundraising" themes.
k=11: This k value demonstrates the best balance in this new set of runs.
It produces several very distinct and interpretable financial/business topics: "Wealth Management/European Footprint" , "Severance/Headcount/Capital" , "Credit Card/Consumer" , "GSIB/Allocation" , "Bank/Acquisition/Advisory" , "FICC/Equity/Commodity" , and "Deposit/Capital/Market/Exposure".

Crucially, the "Apple" anomaly is not present in the top words of any topic for k=11, suggesting a cleaner separation of terms.
While some conversational noise is still present (Topics 2, 4, 9 in k=11), the quality of the interpretable topics is high.
k=12, k=13, k=14: Beyond k=11, the topics generally become more fragmented, or reintroduce the "Apple" anomaly, and the number of less coherent/conversational topics increases, making overall interpretation more challenging. For example, k=12 recombines "funding/deposits" with "severance/headcount", which is less ideal.
Therefore, my strongest recommendation is k=11. It provides a good level of detail for key financial aspects of Goldman Sachs' earnings calls while offering significantly improved topic coherence and distinctiveness, and effectively mitigating some of the persistent noise terms seen in other k values.

In [ ]:
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer # Assuming this is already imported and used
import pandas as pd # Assuming grouped_gs_qna_df is a pandas DataFrame

def get_lda_topic_model_and_data(dtm, vectorizer, num_topics=11, no_top_words=10, **lda_params):
    """
    Initializes and fits an LDA model, then extracts top words and their weights for each topic.

    Args:
        dtm (scipy.sparse.csr_matrix): The Document-Term Matrix.
        vectorizer (TfidfVectorizer): The fitted TF-IDF vectorizer.
        num_topics (int): The number of topics for the LDA model.
        no_top_words (int): The number of top words to display for each topic.
        **lda_params: Arbitrary keyword arguments to pass to the LatentDirichletAllocation constructor.
                      Common parameters include max_iter, learning_method, learning_decay,
                      random_state, n_jobs, evaluate_every.

    Returns:
        tuple: A tuple containing:
            - lda_model (LatentDirichletAllocation): The fitted LDA model.
            - topics_data (list): A list of dictionaries, one for each topic,
                                  containing 'topic_idx', 'top_words', and 'word_weights'.
    """
    print(f"\nStarting LDA model fitting with {num_topics} topics...")

    default_lda_params = {
        'max_iter': 10,
        'learning_method': 'online',
        'learning_decay': 0.7,
        'random_state': 42,
        'n_jobs': -1,
        'evaluate_every': 10
    }
    effective_lda_params = {**default_lda_params, **lda_params}

    lda_model = LatentDirichletAllocation(n_components=num_topics, **effective_lda_params)
    lda_model.fit(dtm)
    print("LDA model fitting complete.")

    feature_names = vectorizer.get_feature_names_out()
    topics_data = []
    for topic_idx, topic in enumerate(lda_model.components_):
        top_words_indices = topic.argsort()[:-no_top_words - 1:-1]
        top_words = [feature_names[i] for i in top_words_indices]
        word_weights = [topic[i] for i in top_words_indices]

        topics_data.append({
            'topic_idx': topic_idx,
            'top_words': top_words,
            'word_weights': word_weights
        })
    return lda_model, topics_data

def assign_dominant_topics_to_dataframe(df, lda_model, dtm, column_prefix='dominant_topic'):
    """
    Assigns the dominant topic to each document in a DataFrame based on an LDA model.

    Args:
        df (pd.DataFrame): The DataFrame to which dominant topics will be assigned.
        lda_model (LatentDirichletAllocation): The fitted LDA model.
        dtm (scipy.sparse.csr_matrix): The Document-Term Matrix used for fitting the LDA model.
        column_prefix (str): Prefix for the new column name (e.g., 'dominant_topic_k11').
    """
    topic_distribution = lda_model.transform(dtm)
    num_topics = lda_model.n_components
    df[f'{column_prefix}_k{num_topics}'] = np.argmax(topic_distribution, axis=1)
    print(f"Dominant topics assigned to DataFrame under column '{column_prefix}_k{num_topics}'.")


# Phase 3: Vectorization (using the previous function)
dtm_matrix, tfidf_vectorizer = vectorize_text(grouped_gs_qna_df['preprocessed_content'])

# Phase 4: Topic Modeling (using the new improved function)
chosen_num_topics = 11 # From previous analysis, or experiment here
top_words_count = 10

# You can pass custom LDA parameters if needed, e.g., max_iter=20, learning_decay=0.9
lda_model_fitted, topics_data_extracted = get_lda_topic_model_and_data(
    dtm=dtm_matrix,
    vectorizer=tfidf_vectorizer,
    num_topics=chosen_num_topics,
    no_top_words=top_words_count,
    random_state=42 # Explicitly passing for clarity, though it's a default
)

# Print extracted topics data
print("\n--- Extracted Topics Data ---")
for topic in topics_data_extracted:
    print(f"Topic {topic['topic_idx'] + 1}: {' '.join(topic['top_words'])}")

# Assign dominant topics back to the DataFrame
assign_dominant_topics_to_dataframe(grouped_gs_qna_df, lda_model_fitted, dtm_matrix)

print("\nProcessing complete. The DataFrame now contains dominant topics.")

In [ ]:
# Updated DataFrame with dominant topics (first 5 rows)
grouped_gs_qna_df.head()

In [ ]:
for topic in topics_data_extracted:
    print(f"\nTopic {topic['topic_idx'] + 1}:")
    for word, weight in zip(topic['top_words'], topic['word_weights']):
        print(f"{word} (weight: {weight:.4f})")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
topic_labels_map = {
    0: "Strategic Positioning & Platform",
    1: "Wealth Management & European Markets",
    2: "Alternative Investments & Fee Income",
    3: "Headcount & Workforce Management",
    4: "Consumer Credit & Card Performance",
    5: "FICC & Equity Trading Performance",
    6: "Regulatory Capital & Institutional Allocation",
    7: "Client-Centric Growth & Solutions",
    8: "M&A, Valuations & Advisory",
    9: "FICC & Market Environment",
    10: "Deposits, Capital & Funding"
}

# Assign labels to the topics_data
for topic_info in topics_data:
    topic_info['label'] = topic_labels_map.get(topic_info['topic_idx'], f"Unlabeled Topic {topic_info['topic_idx']}")

print("\n--- Topics with assigned labels and top words ---")
for topic_info in topics_data:
    print(f"Topic {topic_info['topic_idx'] + 1}: {topic_info['label']}")
    print(f"  Top Words: {' '.join(topic_info['top_words'])}")
    print("-" * 30)

# --- Visual Display of Topics ---
print("\n--- Generating visual display of topics ---")

n_cols = 3
n_rows = (num_topics + n_cols - 1) // n_cols
plt.figure(figsize=(n_cols * 6, n_rows * 4), dpi=100)

for i, topic_info in enumerate(topics_data):
    ax = plt.subplot(n_rows, n_cols, i + 1)
    df_plot = pd.DataFrame({
        'word': topic_info['top_words'],
        'weight': topic_info['word_weights']
    })
    df_plot = df_plot.sort_values(by='weight', ascending=True)
    sns.barplot(x='weight', y='word', data=df_plot, palette='magma', ax=ax)
    ax.set_title(f"{topic_info['label']}", fontsize=11, fontweight='bold', pad=10)
    ax.set_xlabel("Word Importance (Weight)", fontsize=9)
    ax.set_ylabel("")
    ax.tick_params(axis='both', which='major', labelsize=8)
    sns.despine(ax=ax, top=True, right=True, left=False, bottom=False)
    ax.tick_params(axis='y', length=0)

plt.tight_layout(rect=[0, 0, 1, 0.98])
plt.suptitle(
    f"Top Words for {num_topics} Topics in Goldman Sachs Earnings Calls (Q&A Section)",
    y=1.00, fontsize=16, fontweight='bold'
)
plt.show()

print("\nVisual display generated. Please review the plots and verify the topic labels.")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.countplot(
    data=grouped_gs_qna_df,
    x='year',
    hue='quarter',
    palette='tab10'
)
plt.xlabel('Year')
plt.ylabel('Number of Documents')
plt.title('Distribution of Documents by Year and Quarter\nGoldman Sachs Q&A')
plt.legend(title='Quarter')
plt.tight_layout()
plt.show()

# Now, for dominant topics over both year and quarter:
plt.figure(figsize=(16, 7))
sns.countplot(
    data=grouped_gs_qna_df,
    x='dominant_topic_k11',
    hue='year',
    palette='tab10'
)
topic_labels = [topic_labels_map.get(i, f"Topic {i}") for i in sorted(grouped_gs_qna_df['dominant_topic_k11'].unique())]
plt.xticks(ticks=range(len(topic_labels)), labels=topic_labels, rotation=45, ha='right')
plt.xlabel('Dominant Topic (k=11)')
plt.ylabel('Number of Documents')
plt.title('Dominant Topics (k=11) by Year\nGoldman Sachs Q&A')
plt.legend(title='Year')
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

years = sorted(grouped_gs_qna_df['year'].unique())
n_years = len(years)

fig, axes = plt.subplots(n_years, 1, figsize=(10, 4 * n_years), sharex=True)

if n_years == 1:
    axes = [axes]

for idx, year in enumerate(years):
    ax = axes[idx]
    data = grouped_gs_qna_df[grouped_gs_qna_df['year'] == year]
    sns.countplot(
        data=data,
        x='quarter',
        hue='dominant_topic_k11',
        palette='tab10',
        ax=ax
    )
    ax.set_title(f'Distribution of Dominant Topics (k=11) by Quarter - {year}')
    ax.set_xlabel('Quarter')
    ax.set_ylabel('Number of Documents')
    ax.legend(
        title='Dominant Topic',
        loc='upper right',
        labels=[topic_labels_map.get(i, f"Topic {i}") for i in sorted(data['dominant_topic_k11'].unique())]
    )

# Fix color mapping so each topic always has the same color across years
unique_topics = sorted(grouped_gs_qna_df['dominant_topic_k11'].unique())
topic_palette = sns.color_palette('tab10', n_colors=len(unique_topics))
topic_color_dict = {topic: topic_palette[i % len(topic_palette)] for i, topic in enumerate(unique_topics)}

for idx, year in enumerate(years):
    ax = axes[idx]
    data = grouped_gs_qna_df[grouped_gs_qna_df['year'] == year]
    # Use the same color mapping for all years
    sns.countplot(
        data=data,
        x='quarter',
        hue='dominant_topic_k11',
        palette=topic_color_dict,
        ax=ax
    )
    ax.set_title(f'Distribution of Dominant Topics (k=11) by Quarter - {year}')
    ax.set_xlabel('Quarter')
    ax.set_ylabel('Number of Documents')
    handles, labels = ax.get_legend_handles_labels()
    # Always use the same order and labels for legend
    ordered_labels = [topic_labels_map.get(t, f"Topic {t}") for t in unique_topics]
    ax.legend(handles, ordered_labels, title='Dominant Topic', loc='upper right')

plt.suptitle(
    "Quarterly Distribution of Dominant Topics (k=11) by Year\nGoldman Sachs Earnings Call Transcript (Q&A Section)",
    fontsize=16, fontweight='bold', y=1.02
)
plt.tight_layout()
plt.show()

# Save Data Example

In [ ]:
import pandas as pd

target_dir = 'data/temp/'
file_name = 'dummy_test_output_new.csv'
dummy_pf = pd.DataFrame({'from_colab': [IS_COLAB, True, 'hello']})


if OUTPUT_PROCESSED_FILES:
    if IS_COLAB:
        AUTHENTICATED_REPO_URL = REPO_URL.replace("https://", f"https://{GITHUB_USERNAME}:{GITHUB_TOKEN}@")
        dummy_pf.to_csv(f"{target_dir}{file_name}", index=False)

        # Configure Git user (important for committing)
        !git config user.email "{GITHUB_EMAIL}"
        !git config user.name "{GITHUB_USERNAME}"
        !git remote set-url origin {AUTHENTICATED_REPO_URL}

        # Add the file to staging
        !git add {target_dir}{file_name}
        print(f"Added '{target_dir}{file_name}' to staging.")

        # Commit the changes
        commit_message = f"Add new data file: {target_dir}{file_name}"
        !git commit -m "{commit_message}"
        print(f"Committed changes with message: '{commit_message}'")
        print(f"Attempted commit with message: '{commit_message}'")

        # Add this line to debug:
        print(f"Value of REPO_BRANCH before push: {REPO_BRANCH}")

        print("Pushing changes to GitHub. Please enter your GitHub username and Personal Access Token when prompted.")
        !git push --set-upstream origin {REPO_BRANCH} --force
        print("Push command executed. Check output for success or prompt.")
    else:
        dummy_pf.to_csv(f"{target_dir}{file_name}", index=False)
        print("Processed files saved successfully.")